# 🎯 Demonstração Prática - Análise Preditiva E-commerce

## Sistema de Recomendação de Produtos com MongoDB + PostgreSQL

Este notebook demonstra uma aplicação prática de análise preditiva para um sistema de recomendação de produtos e-commerce, utilizando uma arquitetura híbrida com MongoDB e PostgreSQL.

### 📋 Objetivos da Demonstração:
- **Análise Descritiva:** Entender o comportamento dos usuários e produtos
- **Análise Preditiva:** Predizer probabilidade de compra e churn
- **Clustering:** Segmentar usuários por comportamento
- **Recomendações:** Gerar recomendações personalizadas
- **Visualizações:** Criar dashboards interativos

### 🛠️ Tecnologias Utilizadas:
- **MongoDB:** Dados não estruturados e comportamento
- **PostgreSQL:** Dados transacionais e relatórios
- **Python:** Análise de dados e machine learning
- **Scikit-learn:** Algoritmos de ML
- **Matplotlib/Seaborn:** Visualizações
- **Pandas:** Manipulação de dados

---


In [ ]:
# 📦 Importar Bibliotecas Necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Bancos de dados
from pymongo import MongoClient
import psycopg2
from psycopg2.extras import RealDictCursor

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Bibliotecas importadas com sucesso!")
print("🚀 Pronto para iniciar a demonstração!")


## 🗄️ Parte 1: Conexão com Bancos de Dados

### MongoDB - Dados Não Estruturados
MongoDB será usado para armazenar:
- **Produtos:** Características flexíveis e reviews
- **Comportamento:** Eventos de navegação e interação
- **Recomendações:** Scores e matrizes de similaridade

### PostgreSQL - Dados Transacionais
PostgreSQL será usado para:
- **Usuários:** Dados pessoais e segmentação
- **Pedidos:** Transações e histórico de compras
- **Relatórios:** Análises estruturadas e KPIs


In [ ]:
# 🔌 Conectar ao MongoDB
try:
    mongo_client = MongoClient('mongodb://localhost:27017')
    mongo_db = mongo_client['ecommerce_demo']
    print("✅ Conectado ao MongoDB com sucesso!")
    
    # Testar conexão
    mongo_db.command('ping')
    print("✅ Ping MongoDB: OK")
    
except Exception as e:
    print(f"❌ Erro ao conectar MongoDB: {e}")
    print("💡 Certifique-se de que o MongoDB está rodando")
    mongo_client = None


In [ ]:
# 🔌 Conectar ao PostgreSQL
try:
    postgres_conn = psycopg2.connect(
        host='localhost',
        database='ecommerce_demo',
        user='postgres',
        password='postgres'
    )
    postgres_cursor = postgres_conn.cursor(cursor_factory=RealDictCursor)
    print("✅ Conectado ao PostgreSQL com sucesso!")
    
    # Testar conexão
    postgres_cursor.execute("SELECT version();")
    version = postgres_cursor.fetchone()
    print(f"✅ PostgreSQL Version: {version[0][:50]}...")
    
except Exception as e:
    print(f"❌ Erro ao conectar PostgreSQL: {e}")
    print("💡 Certifique-se de que o PostgreSQL está rodando e o banco existe")
    postgres_conn = None


## 📊 Parte 2: Análise Descritiva - MongoDB

### Objetivo: Entender o comportamento dos usuários através de dados não estruturados

Vamos analisar:
1. **Produtos mais visualizados**
2. **Padrões de navegação**
3. **Eventos de conversão**
4. **Segmentação por comportamento**


In [ ]:
# 📊 Análise de Comportamento - MongoDB
if mongo_client:
    print("🔍 Analisando comportamento dos usuários...")
    
    # Pipeline para análise de comportamento
    pipeline = [
        {"$unwind": "$eventos"},
        {"$group": {
            "_id": "$usuario_id",
            "total_eventos": {"$sum": 1},
            "page_views": {
                "$sum": {"$cond": [{"$eq": ["$eventos.tipo", "page_view"]}, 1, 0]}
            },
            "clicks": {
                "$sum": {"$cond": [{"$eq": ["$eventos.tipo", "click"]}, 1, 0]}
            },
            "add_to_cart": {
                "$sum": {"$cond": [{"$eq": ["$eventos.tipo", "add_to_cart"]}, 1, 0]}
            },
            "searches": {
                "$sum": {"$cond": [{"$eq": ["$eventos.tipo", "search"]}, 1, 0]}
            },
            "tempo_total": {"$sum": "$eventos.tempo_pagina"},
            "produtos_unicos": {"$addToSet": "$eventos.produto_id"}
        }},
        {"$project": {
            "usuario_id": "$_id",
            "total_eventos": 1,
            "page_views": 1,
            "clicks": 1,
            "add_to_cart": 1,
            "searches": 1,
            "tempo_total": 1,
            "produtos_unicos": {"$size": "$produtos_unicos"},
            "taxa_conversao": {
                "$cond": [
                    {"$gt": ["$page_views", 0]},
                    {"$divide": ["$add_to_cart", "$page_views"]},
                    0
                ]
            },
            "tempo_medio_evento": {
                "$cond": [
                    {"$gt": ["$total_eventos", 0]},
                    {"$divide": ["$tempo_total", "$total_eventos"]},
                    0
                ]
            }
        }},
        {"$sort": {"total_eventos": -1}}
    ]
    
    # Executar pipeline
    comportamento_data = list(mongo_db.usuarios_comportamento.aggregate(pipeline))
    
    if comportamento_data:
        comportamento_df = pd.DataFrame(comportamento_data)
        print(f"✅ Analisados {len(comportamento_df)} usuários")
        print("\n📈 Resumo do Comportamento:")
        print(comportamento_df.describe().round(2))
        
        # Mostrar top usuários mais ativos
        print("\n🏆 Top 5 Usuários Mais Ativos:")
        top_usuarios = comportamento_df.head()
        for _, user in top_usuarios.iterrows():
            print(f"  {user['usuario_id']}: {user['total_eventos']} eventos, "
                  f"{user['taxa_conversao']:.1%} conversão")
    else:
        print("❌ Nenhum dado de comportamento encontrado")
        comportamento_df = None
else:
    print("❌ MongoDB não disponível - usando dados simulados")
    # Criar dados simulados para demonstração
    np.random.seed(42)
    comportamento_df = pd.DataFrame({
        'usuario_id': [f'U{i:03d}' for i in range(1, 21)],
        'total_eventos': np.random.poisson(15, 20),
        'page_views': np.random.poisson(10, 20),
        'clicks': np.random.poisson(8, 20),
        'add_to_cart': np.random.poisson(2, 20),
        'searches': np.random.poisson(3, 20),
        'produtos_unicos': np.random.randint(1, 8, 20),
        'taxa_conversao': np.random.beta(2, 8, 20),
        'tempo_medio_evento': np.random.normal(45, 15, 20)
    })
    print(f"✅ Criados {len(comportamento_df)} usuários simulados")


## 🎯 Parte 3: Análise Preditiva - Clustering de Usuários

### Objetivo: Segmentar usuários por comportamento usando K-Means

Vamos identificar:
1. **Usuários Ativos e Convertidos** 🔥
2. **Usuários Ativos mas Baixa Conversão** 👀  
3. **Usuários Passivos** 😴


In [ ]:
# 🎯 Clustering de Usuários com K-Means
if comportamento_df is not None and not comportamento_df.empty:
    print("🔍 Aplicando clustering K-Means...")
    
    # Preparar features para clustering
    features = ['total_eventos', 'page_views', 'clicks', 'add_to_cart', 'produtos_unicos', 'taxa_conversao']
    X = comportamento_df[features].fillna(0)
    
    # Normalizar dados
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Aplicar K-Means
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    comportamento_df['cluster'] = kmeans.fit_predict(X_scaled)
    
    # Análise dos clusters
    cluster_analysis = comportamento_df.groupby('cluster')[features].mean()
    
    print("📊 Análise de Clusters:")
    print(cluster_analysis.round(2))
    
    # Interpretar clusters
    print("\n🏷️ Interpretação dos Clusters:")
    cluster_names = {
        0: "😴 Usuários Passivos",
        1: "👀 Usuários Ativos mas Baixa Conversão", 
        2: "🔥 Usuários Ativos e Convertidos"
    }
    
    for cluster_id in range(3):
        cluster_data = comportamento_df[comportamento_df['cluster'] == cluster_id]
        avg_events = cluster_data['total_eventos'].mean()
        avg_conversion = cluster_data['taxa_conversao'].mean()
        
        print(f"\n{cluster_names[cluster_id]}:")
        print(f"  Usuários: {len(cluster_data)}")
        print(f"  Eventos médios: {avg_events:.1f}")
        print(f"  Taxa conversão: {avg_conversion:.1%}")
        print(f"  Produtos únicos: {cluster_data['produtos_unicos'].mean():.1f}")
        
        # Mostrar alguns usuários do cluster
        sample_users = cluster_data['usuario_id'].head(3).tolist()
        print(f"  Exemplos: {', '.join(sample_users)}")
    
    print(f"\n✅ Clustering concluído! {len(comportamento_df)} usuários segmentados em 3 grupos")
else:
    print("❌ Dados de comportamento não disponíveis para clustering")


## 📈 Parte 4: Análise Preditiva - PostgreSQL

### Objetivo: Predizer churn de usuários baseado em dados transacionais

Vamos analisar:
1. **Histórico de compras**
2. **Padrões de gastos**
3. **Frequência de compras**
4. **Probabilidade de churn**


In [ ]:
# 📈 Análise Preditiva - PostgreSQL
if postgres_conn:
    print("🔍 Analisando dados transacionais...")
    
    # Query para análise de usuários
    query = """
        SELECT 
            u.usuario_id,
            u.nome,
            u.segmento,
            u.valor_total_compras,
            COUNT(p.id) as total_pedidos,
            AVG(p.valor_total) as ticket_medio,
            COUNT(DISTINCT ip.produto_id) as produtos_unicos,
            MAX(p.data_pedido) as ultima_compra,
            EXTRACT(DAYS FROM NOW() - MAX(p.data_pedido)) as dias_sem_comprar,
            COUNT(CASE WHEN p.status = 'concluido' THEN 1 END) as pedidos_concluidos,
            COUNT(CASE WHEN p.status = 'pendente' THEN 1 END) as pedidos_pendentes,
            STDDEV(p.valor_total) as variabilidade_gastos
        FROM usuarios u
        LEFT JOIN pedidos p ON u.id = p.usuario_id
        LEFT JOIN itens_pedido ip ON p.id = ip.pedido_id
        GROUP BY u.usuario_id, u.nome, u.segmento, u.valor_total_compras
        ORDER BY u.valor_total_compras DESC
    """
    
    postgres_cursor.execute(query)
    usuarios_transacionais = pd.DataFrame(postgres_cursor.fetchall())
    
    if not usuarios_transacionais.empty:
        print(f"✅ Analisados {len(usuarios_transacionais)} usuários transacionais")
        
        # Preparar dados para predição de churn
        usuarios_transacionais['dias_sem_comprar'] = usuarios_transacionais['dias_sem_comprar'].fillna(365)
        usuarios_transacionais['ticket_medio'] = usuarios_transacionais['ticket_medio'].fillna(0)
        usuarios_transacionais['produtos_unicos'] = usuarios_transacionais['produtos_unicos'].fillna(0)
        usuarios_transacionais['variabilidade_gastos'] = usuarios_transacionais['variabilidade_gastos'].fillna(0)
        
        # Criar variável target (churn = dias sem comprar > 30)
        usuarios_transacionais['churn'] = (usuarios_transacionais['dias_sem_comprar'] > 30).astype(int)
        
        print("\n📊 Estatísticas dos Usuários:")
        print(f"  Valor médio de compras: R$ {usuarios_transacionais['valor_total_compras'].mean():,.2f}")
        print(f"  Pedidos médios por usuário: {usuarios_transacionais['total_pedidos'].mean():.1f}")
        print(f"  Ticket médio: R$ {usuarios_transacionais['ticket_medio'].mean():,.2f}")
        print(f"  Taxa de churn: {usuarios_transacionais['churn'].mean():.1%}")
        
        # Mostrar usuários com maior risco de churn
        risco_churn = usuarios_transacionais[usuarios_transacionais['churn'] == 1].sort_values('dias_sem_comprar', ascending=False)
        print(f"\n⚠️ Usuários com Risco de Churn ({len(risco_churn)} usuários):")
        for _, user in risco_churn.head(5).iterrows():
            print(f"  {user['nome']}: {user['dias_sem_comprar']:.0f} dias sem comprar, "
                  f"R$ {user['valor_total_compras']:,.2f} histórico")
        
    else:
        print("❌ Nenhum dado transacional encontrado")
        usuarios_transacionais = None
else:
    print("❌ PostgreSQL não disponível - usando dados simulados")
    # Criar dados simulados para demonstração
    np.random.seed(42)
    usuarios_transacionais = pd.DataFrame({
        'usuario_id': [f'U{i:03d}' for i in range(1, 21)],
        'nome': [f'Usuário {i}' for i in range(1, 21)],
        'segmento': np.random.choice(['high_value', 'medium_value', 'low_value', 'new_user'], 20),
        'valor_total_compras': np.random.exponential(2000, 20),
        'total_pedidos': np.random.poisson(5, 20),
        'ticket_medio': np.random.normal(500, 200, 20),
        'produtos_unicos': np.random.randint(1, 10, 20),
        'dias_sem_comprar': np.random.exponential(30, 20),
        'pedidos_concluidos': np.random.poisson(4, 20),
        'pedidos_pendentes': np.random.poisson(1, 20),
        'variabilidade_gastos': np.random.exponential(100, 20)
    })
    usuarios_transacionais['churn'] = (usuarios_transacionais['dias_sem_comprar'] > 30).astype(int)
    print(f"✅ Criados {len(usuarios_transacionais)} usuários transacionais simulados")


In [ ]:
# 🎯 Modelo de Predição de Churn
if usuarios_transacionais is not None and not usuarios_transacionais.empty:
    print("🤖 Treinando modelo de predição de churn...")
    
    # Features para o modelo
    features = ['valor_total_compras', 'total_pedidos', 'ticket_medio', 'produtos_unicos', 'dias_sem_comprar', 'variabilidade_gastos']
    X = usuarios_transacionais[features].fillna(0)
    y = usuarios_transacionais['churn']
    
    # Dividir dados
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Treinar modelo Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)
    model.fit(X_train, y_train)
    
    # Fazer predições
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Avaliar modelo
    print("📈 Relatório de Classificação:")
    print(classification_report(y_test, y_pred))
    
    # Importância das features
    feature_importance = pd.DataFrame({
        'feature': features,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n🔍 Importância das Features:")
    print(feature_importance.to_string(index=False))
    
    # Predições para todos os usuários
    usuarios_transacionais['probabilidade_churn'] = model.predict_proba(X)[:, 1]
    usuarios_transacionais['predicao_churn'] = model.predict(X)
    
    # Mostrar usuários com maior probabilidade de churn
    risco_alto = usuarios_transacionais[usuarios_transacionais['probabilidade_churn'] > 0.7].sort_values('probabilidade_churn', ascending=False)
    print(f"\n🚨 Usuários com Alto Risco de Churn ({len(risco_alto)} usuários):")
    for _, user in risco_alto.head(5).iterrows():
        print(f"  {user['nome']}: {user['probabilidade_churn']:.1%} probabilidade, "
              f"{user['dias_sem_comprar']:.0f} dias sem comprar")
    
    print(f"\n✅ Modelo treinado com {len(X_train)} amostras de treino e {len(X_test)} de teste")
else:
    print("❌ Dados transacionais não disponíveis para predição de churn")


## 📊 Parte 5: Visualizações Interativas

### Objetivo: Criar dashboards visuais para análise dos resultados

Vamos criar:
1. **Gráficos de comportamento**
2. **Análise de clusters**
3. **Predições de churn**
4. **Comparação entre bancos**


In [ ]:
# 📊 Criar Visualizações Interativas
print("🎨 Criando visualizações...")

# Configurar figura com subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('🎯 Dashboard - Análise Preditiva E-commerce', fontsize=16, fontweight='bold')

# Gráfico 1: Distribuição de eventos por usuário (MongoDB)
if comportamento_df is not None and not comportamento_df.empty:
    axes[0, 0].hist(comportamento_df['total_eventos'], bins=15, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('📱 Distribuição de Eventos por Usuário', fontweight='bold')
    axes[0, 0].set_xlabel('Total de Eventos')
    axes[0, 0].set_ylabel('Frequência')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Adicionar estatísticas
    mean_events = comportamento_df['total_eventos'].mean()
    axes[0, 0].axvline(mean_events, color='red', linestyle='--', linewidth=2, label=f'Média: {mean_events:.1f}')
    axes[0, 0].legend()

# Gráfico 2: Taxa de conversão vs Page Views
if comportamento_df is not None and not comportamento_df.empty:
    scatter = axes[0, 1].scatter(comportamento_df['page_views'], comportamento_df['taxa_conversao'], 
                               alpha=0.7, s=100, c=comportamento_df['total_eventos'], 
                               cmap='viridis', edgecolors='black')
    axes[0, 1].set_title('🎯 Taxa de Conversão vs Page Views', fontweight='bold')
    axes[0, 1].set_xlabel('Page Views')
    axes[0, 1].set_ylabel('Taxa de Conversão')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Adicionar colorbar
    cbar = plt.colorbar(scatter, ax=axes[0, 1])
    cbar.set_label('Total de Eventos')

# Gráfico 3: Clusters de usuários
if comportamento_df is not None and 'cluster' in comportamento_df.columns:
    cluster_colors = {0: 'lightcoral', 1: 'gold', 2: 'lightgreen'}
    cluster_labels = {0: 'Passivos', 1: 'Ativos Baixa Conv.', 2: 'Ativos Convertidos'}
    
    for cluster_id in range(3):
        cluster_data = comportamento_df[comportamento_df['cluster'] == cluster_id]
        axes[0, 2].scatter(cluster_data['total_eventos'], cluster_data['taxa_conversao'],
                          c=cluster_colors[cluster_id], label=cluster_labels[cluster_id],
                          alpha=0.7, s=100, edgecolors='black')
    
    axes[0, 2].set_title('🎯 Clusters de Usuários', fontweight='bold')
    axes[0, 2].set_xlabel('Total de Eventos')
    axes[0, 2].set_ylabel('Taxa de Conversão')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)

# Gráfico 4: Distribuição de valores de compra (PostgreSQL)
if usuarios_transacionais is not None and not usuarios_transacionais.empty:
    axes[1, 0].hist(usuarios_transacionais['valor_total_compras'], bins=15, alpha=0.7, 
                   color='lightgreen', edgecolor='black')
    axes[1, 0].set_title('💰 Distribuição de Valores de Compra', fontweight='bold')
    axes[1, 0].set_xlabel('Valor Total Compras (R$)')
    axes[1, 0].set_ylabel('Frequência')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Adicionar estatísticas
    mean_value = usuarios_transacionais['valor_total_compras'].mean()
    axes[1, 0].axvline(mean_value, color='red', linestyle='--', linewidth=2, label=f'Média: R$ {mean_value:,.0f}')
    axes[1, 0].legend()

# Gráfico 5: Segmentos de usuários
if usuarios_transacionais is not None and not usuarios_transacionais.empty:
    segmentos = usuarios_transacionais['segmento'].value_counts()
    colors = ['gold', 'lightblue', 'lightcoral', 'lightgreen']
    wedges, texts, autotexts = axes[1, 1].pie(segmentos.values, labels=segmentos.index, 
                                            autopct='%1.1f%%', startangle=90, colors=colors)
    axes[1, 1].set_title('👥 Distribuição por Segmento', fontweight='bold')
    
    # Melhorar legibilidade
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')

# Gráfico 6: Probabilidade de churn
if usuarios_transacionais is not None and 'probabilidade_churn' in usuarios_transacionais.columns:
    # Criar bins para probabilidade de churn
    bins = [0, 0.3, 0.7, 1.0]
    labels = ['Baixo Risco', 'Médio Risco', 'Alto Risco']
    usuarios_transacionais['risco_churn'] = pd.cut(usuarios_transacionais['probabilidade_churn'], 
                                                  bins=bins, labels=labels, include_lowest=True)
    
    risco_counts = usuarios_transacionais['risco_churn'].value_counts()
    colors = ['lightgreen', 'gold', 'lightcoral']
    
    bars = axes[1, 2].bar(risco_counts.index, risco_counts.values, color=colors, alpha=0.7, edgecolor='black')
    axes[1, 2].set_title('🚨 Distribuição de Risco de Churn', fontweight='bold')
    axes[1, 2].set_ylabel('Número de Usuários')
    axes[1, 2].grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for bar in bars:
        height = bar.get_height()
        axes[1, 2].text(bar.get_x() + bar.get_width()/2., height + 0.1,
                       f'{int(height)}', ha='center', va='bottom', fontweight='bold')

# Ajustar layout
plt.tight_layout()

# Salvar gráfico
plt.savefig('dashboard_analise_preditiva.png', dpi=300, bbox_inches='tight')
print("✅ Dashboard salvo como 'dashboard_analise_preditiva.png'")

# Mostrar gráfico
plt.show()

print("\n🎉 Visualizações criadas com sucesso!")
print("📊 Dashboard completo com análises de MongoDB e PostgreSQL")


## 🎯 Parte 6: Sistema de Recomendações

### Objetivo: Gerar recomendações personalizadas baseadas na análise preditiva

Vamos implementar:
1. **Algoritmo colaborativo simples**
2. **Scoring baseado em comportamento**
3. **Recomendações por cluster**
4. **Integração MongoDB + PostgreSQL**


In [ ]:
# 🎯 Sistema de Recomendações
print("🎯 Gerando recomendações personalizadas...")

def gerar_recomendacoes(usuario_id, comportamento_df, usuarios_transacionais):
    """Gerar recomendações para um usuário específico"""
    
    # Produtos disponíveis (simulados)
    produtos_disponiveis = [
        {'id': 'P001', 'nome': 'Smartphone Galaxy S24', 'categoria': 'smartphones', 'preco': 2999.99},
        {'id': 'P002', 'nome': 'iPhone 15 Pro', 'categoria': 'smartphones', 'preco': 8999.99},
        {'id': 'P003', 'nome': 'Notebook Dell XPS 13', 'categoria': 'notebooks', 'preco': 5999.99},
        {'id': 'P004', 'nome': 'Tablet iPad Air', 'categoria': 'tablets', 'preco': 3999.99},
        {'id': 'P005', 'nome': 'Smartphone Xiaomi 13', 'categoria': 'smartphones', 'preco': 1999.99},
        {'id': 'P006', 'nome': 'Notebook MacBook Air', 'categoria': 'notebooks', 'preco': 7999.99},
        {'id': 'P007', 'nome': 'Tablet Samsung Galaxy Tab', 'categoria': 'tablets', 'preco': 2499.99},
        {'id': 'P008', 'nome': 'Smartphone Pixel 8', 'categoria': 'smartphones', 'preco': 3499.99}
    ]
    
    # Buscar dados do usuário
    user_comportamento = comportamento_df[comportamento_df['usuario_id'] == usuario_id] if comportamento_df is not None else None
    user_transacional = usuarios_transacionais[usuarios_transacionais['usuario_id'] == usuario_id] if usuarios_transacionais is not None else None
    
    recomendacoes = []
    
    for produto in produtos_disponiveis:
        score = 0.0
        motivos = []
        
        # Score baseado em comportamento (MongoDB)
        if user_comportamento is not None and not user_comportamento.empty:
            user_data = user_comportamento.iloc[0]
            
            # Usuários ativos têm preferência por produtos similares
            if user_data['total_eventos'] > comportamento_df['total_eventos'].mean():
                score += 0.3
                motivos.append("usuário_ativo")
            
            # Usuários com alta conversão preferem produtos premium
            if user_data['taxa_conversao'] > comportamento_df['taxa_conversao'].mean():
                if produto['preco'] > 5000:
                    score += 0.2
                    motivos.append("preferencia_premium")
                else:
                    score += 0.1
                    motivos.append("produto_acessivel")
            
            # Usuários que visualizam muitos produtos têm preferência diversificada
            if user_data['produtos_unicos'] > comportamento_df['produtos_unicos'].mean():
                score += 0.15
                motivos.append("interesse_diversificado")
        
        # Score baseado em dados transacionais (PostgreSQL)
        if user_transacional is not None and not user_transacional.empty:
            user_data = user_transacional.iloc[0]
            
            # Usuários high_value preferem produtos caros
            if user_data['segmento'] == 'high_value':
                if produto['preco'] > 5000:
                    score += 0.4
                    motivos.append("segmento_high_value")
                else:
                    score += 0.1
                    motivos.append("produto_economico")
            
            # Usuários com muitos pedidos preferem produtos populares
            if user_data['total_pedidos'] > usuarios_transacionais['total_pedidos'].mean():
                score += 0.2
                motivos.append("cliente_frequente")
            
            # Usuários com baixo risco de churn têm preferência por produtos similares
            if 'probabilidade_churn' in user_data and user_data['probabilidade_churn'] < 0.3:
                score += 0.15
                motivos.append("baixo_risco_churn")
        
        # Score baseado em cluster (se disponível)
        if user_comportamento is not None and 'cluster' in user_comportamento.columns:
            cluster = user_comportamento.iloc[0]['cluster']
            
            if cluster == 2:  # Usuários ativos e convertidos
                score += 0.25
                motivos.append("cluster_ativo_convertido")
            elif cluster == 1:  # Usuários ativos mas baixa conversão
                if produto['preco'] < 3000:  # Produtos mais baratos
                    score += 0.2
                    motivos.append("cluster_ativo_baixa_conv")
            elif cluster == 0:  # Usuários passivos
                if produto['preco'] < 2000:  # Produtos muito baratos
                    score += 0.15
                    motivos.append("cluster_passivo")
        
        # Adicionar score baseado em categoria (simulação de preferência)
        categoria_scores = {'smartphones': 0.1, 'notebooks': 0.05, 'tablets': 0.08}
        score += categoria_scores.get(produto['categoria'], 0.02)
        
        # Adicionar ruído aleatório para simular variação
        score += random.uniform(-0.1, 0.1)
        score = max(0, min(1, score))  # Manter entre 0 e 1
        
        recomendacoes.append({
            'produto_id': produto['id'],
            'nome': produto['nome'],
            'categoria': produto['categoria'],
            'preco': produto['preco'],
            'score': score,
            'motivos': motivos
        })
    
    # Ordenar por score e retornar top 5
    recomendacoes.sort(key=lambda x: x['score'], reverse=True)
    return recomendacoes[:5]

# Gerar recomendações para alguns usuários
usuarios_exemplo = ['U001', 'U002', 'U003', 'U004', 'U005']

print("🎯 Recomendações Personalizadas:")
print("=" * 60)

for usuario in usuarios_exemplo:
    print(f"\n👤 Usuário: {usuario}")
    
    # Buscar informações do usuário
    user_info = ""
    if comportamento_df is not None:
        user_comp = comportamento_df[comportamento_df['usuario_id'] == usuario]
        if not user_comp.empty:
            user_info += f"Eventos: {user_comp.iloc[0]['total_eventos']}, "
            user_info += f"Conversão: {user_comp.iloc[0]['taxa_conversao']:.1%}"
            if 'cluster' in user_comp.columns:
                cluster_names = {0: 'Passivo', 1: 'Ativo Baixa Conv.', 2: 'Ativo Convertido'}
                user_info += f", Cluster: {cluster_names.get(user_comp.iloc[0]['cluster'], 'N/A')}"
    
    if usuarios_transacionais is not None:
        user_trans = usuarios_transacionais[usuarios_transacionais['usuario_id'] == usuario]
        if not user_trans.empty:
            user_info += f", Segmento: {user_trans.iloc[0]['segmento']}"
            if 'probabilidade_churn' in user_trans.columns:
                user_info += f", Risco Churn: {user_trans.iloc[0]['probabilidade_churn']:.1%}"
    
    print(f"📊 Perfil: {user_info}")
    
    # Gerar recomendações
    recomendacoes = gerar_recomendacoes(usuario, comportamento_df, usuarios_transacionais)
    
    print("🎯 Top 3 Recomendações:")
    for i, rec in enumerate(recomendacoes[:3], 1):
        print(f"  {i}. {rec['nome']}")
        print(f"     Preço: R$ {rec['preco']:,.2f}")
        print(f"     Score: {rec['score']:.2f}")
        print(f"     Motivos: {', '.join(rec['motivos'])}")
        print()

print("✅ Sistema de recomendações implementado com sucesso!")
print("🎯 Recomendações baseadas em análise preditiva de MongoDB + PostgreSQL")
